In [1]:
import torch
import sys
sys.path.append('..')
from src.module2graph import GraphInterperterWithGamma
from src.resnet18 import ResNet18
import numpy as np

import graphviz
import itertools
import copy
from torchvision import transforms
import torchvision

import networkx as nx
from tqdm.notebook import tqdm
from typing import Tuple, Dict # actually we don't need it for py>=3.9, but I have 3.8 on my laptop
from src.utils import train_loop, test_loop
#from numba import njit
from src.cifar_data import get_dataloaders


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# forward for target model with gamma values for each edge.
# means - mean values for arguments
def forward_with_gammas(model, gammas: Dict[Tuple[str, str], torch.Tensor], 
                        means: Dict[str, torch.Tensor] = None, *torch_model_args):
    args_iter = iter(torch_model_args)
    env : Dict[str, Node] = {}
    used_edges = set()
    def load_arg(a):    
        return torch.fx.graph.map_arg(a, lambda n: env[n.name])

    def fetch_attr(target : str):
        target_atoms = target.split('.')
        attr_itr = model.graph
        for i, atom in enumerate(target_atoms):
            if not hasattr(attr_itr, atom):
                raise RuntimeError(f"Node referenced nonexistant target {'.'.join(target_atoms[:i])}")
            attr_itr = getattr(attr_itr, atom)
        return attr_itr
    named_modules = dict(model.named_modules())
    for node in model.graph.nodes:
        edges = []

        if node.op in ['call_module', 'call_function', 'output']:    
            if node.op == 'output':
                edges = [(node.args[0][0].name, node.name)]
            else:
    
                for arg in node.args:
                    if type(arg) == torch.fx.Node:  # ignore constants
                        edges.append((arg.name, node.name))
                    else:
                        edges.append(None)
            gammas_node = [int(gammas[e])  if (e is not None) else 1 for e in edges ]
                
            #print (edges, gammas_node)
        if node.op == 'placeholder':
            result = next(args_iter) 
        elif node.op == 'get_attr':
            result = fetch_attr(node.target)
        elif node.op == 'call_function':
            args = [a*g + (1.0 - g) * means[str(a0)] if str(a0) in means else a*g  for a0,a,g in zip(node.args,
                                                                           load_arg(node.args), gammas_node)]
            #print (len(args), len(node.args))
            #print (node, [a for a in node.args])
            #print (node, [a.shape for a in args])
            result = node.target(*args, **load_arg(node.kwargs)) 
        elif node.op == 'call_method':
            self_obj, *args = load_arg(node.args) 
            kwargs = load_arg(node.kwargs)
            args =  [a*g + (1.0 - g) * means[str(a0)] if str(a0) in means else a*g   for a0, a,g in zip(node.args[1:], 
                                                                        args, gammas_node)]
            result = getattr(self_obj, node.target)(*args, **kwargs)
        elif node.op == 'call_module':
            args = [a*g + (1.0 - g) * means[str(a0)] if str(a0) in means else a*g   for a0, a,g in zip(node.args, 
                                                                           load_arg(node.args), gammas_node)]
            
            result = named_modules[node.target](*args, **load_arg(node.kwargs)) 
        
        result = result
        for e in edges:
            used_edges.add(e)
            
        if node.op == 'output':
            
            return result, env # currently ignorign means for output
        #print (node.args, node.name, node.op, abs(result).sum().item())
        env[node.name] = result
        
    return result

# a wrapper that takes model and uses forward_with_Gammas
class PrunedModel(torch.nn.Module):
    def __init__(self, base, prune_dict, means = None):
        super().__init__()
        self.base = base
        self.prune_dict = prune_dict
        self.means = means 
    def forward(self, x):
        return forward_with_gammas(self.base, self.prune_dict,  self.means, x)



In [3]:
# gets intermediate representations of nodes
def get_inter(model, *torch_model_args) -> dict:
    args_iter = iter(torch_model_args)
    env : Dict[str, Node] = {}
    used_edges = set()
    inter = {}
    def load_arg(a):    
        return torch.fx.graph.map_arg(a, lambda n: env[n.name])

    def fetch_attr(target : str):
        target_atoms = target.split('.')
        attr_itr = model.graph
        for i, atom in enumerate(target_atoms):
            if not hasattr(attr_itr, atom):
                raise RuntimeError(f"Node referenced nonexistant target {'.'.join(target_atoms[:i])}")
            attr_itr = getattr(attr_itr, atom)
        return attr_itr
    named_modules = dict(model.named_modules())
    for node in model.graph.nodes:
        edges = []

        if node.op in ['call_module', 'call_function', 'output']:    
            if node.op == 'output':
                edges = [(node.args[0][0].name, node.name)]
            else:
    
                for arg in node.args:
                    if type(arg) == torch.fx.Node:  # ignore constants
                        edges.append((arg.name, node.name))
                    else:
                        edges.append(None)
                
            #print (edges, gammas_node)
        if node.op == 'placeholder':
            result = next(args_iter) 
        elif node.op == 'get_attr':
            result = fetch_attr(node.target)
        elif node.op == 'call_function':
            
            args = load_arg(node.args)
            for a_, a in zip(node.args, args):
                inter[a_] = a
            #print (len(args), len(node.args))
            #print ([a.shape for a in load_arg(node.args)], [a.shape for a in args])
            result = node.target(*args, **load_arg(node.kwargs)) 
        elif node.op == 'call_method':
            self_obj, *args = load_arg(node.args) 
            
            for a_, a in zip(node.args[1:], args):
                inter[a_] = a
            kwargs = load_arg(node.kwargs)
            result = getattr(self_obj, node.target)(*args, **kwargs)
        elif node.op == 'call_module':
            args = load_arg(node.args)
            for a_, a in zip(node.args, args):
                inter[a_] = a
            result = named_modules[node.target](*args, **load_arg(node.kwargs)) 
        
        
        result = result
        for e in edges:
            used_edges.add(e)
    
        if node.op == 'output':
            
            return inter 
        #print (node.args, node.name, node.op, abs(result).sum().item())
        env[node.name] = result
        
    return inter


In [4]:


train_dl, test_dl = get_dataloaders([0,1,2,3,4,5,6,7], )


Files already downloaded and verified
Files already downloaded and verified


In [5]:
from statistics import mean 

def module_to_graph(m: torch.nn.Module):
    graph = torch.fx.symbolic_trace(m).graph
    named_dict = dict(m.named_modules())
    edges = [] # (from, to)
    grad = {'x': 0}
    params = {'x': 0}
    weights = {'x': 0} # node: params
    means = {}
    for node in graph.nodes:
    
        # no placeholder and call_mathod
        if node.op == 'call_module':
            n_params = 0
            grad_1 = []
            params_1 = []
            for p in named_dict[node.target].parameters():
                n_params += p.numel()
                grad_1.append(p.grad)
                params_1.append(p)
            try:
                grad[node.name] = torch.mean(torch.stack(grad_1))
                params[node.name] = torch.mean(torch.stack(params_1))
            except:
                grad[node.name] = 0
                params[node.name] = 0
            weights[node.name] = n_params
            assert len(node.args) == 1
            for arg in node.args:
                if type(arg) == torch.fx.Node:  # ignore constants
                    edges.append((arg.name, node.name))
        elif node.op == 'call_function':
            for arg in node.args:
                if type(arg) == torch.fx.Node:  # ignore constants
                    edges.append((arg.name, node.name))
            weights[node.name] = 0
        elif node.op == 'output':
            try:
                edges.append((node.args[0][0].name, node.name))
            except:
                edges.append((node.args[0].name, node.name))
            weights['output'] = 0
        
        #if  len(edges)>0 and edges[-1] == ('model_maxpool', 'add'):
        #    print (node.args, node.name, node.op)
    return edges, {'_'.join(k.split('.')): v for k, v in weights.items()}, {'_'.join(k.split('.')): v for k, v in params.items()}, {'_'.join(k.split('.')): v for k, v in grad.items()}

module_to_graph(ResNet18())
pass

Using cache found in /Users/konstantinakovlev/.cache/torch/hub/pytorch_vision_v0.10.0
/opt/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/opt/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/opt/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/to

In [6]:
edges, weights, a, b = module_to_graph(ResNet18())
# edges, weights
edges[:10], list(weights.items())[:10]

Using cache found in /Users/konstantinakovlev/.cache/torch/hub/pytorch_vision_v0.10.0


([('x', 'model_conv1'),
  ('model_conv1', 'model_bn1'),
  ('model_bn1', 'model_relu'),
  ('model_relu', 'model_maxpool'),
  ('model_maxpool', 'model_layer1_0_conv1'),
  ('model_layer1_0_conv1', 'model_layer1_0_bn1'),
  ('model_layer1_0_bn1', 'model_layer1_0_relu'),
  ('model_layer1_0_relu', 'model_layer1_0_conv2'),
  ('model_layer1_0_conv2', 'model_layer1_0_bn2'),
  ('model_layer1_0_bn2', 'add')],
 [('x', 0),
  ('model_conv1', 9408),
  ('model_bn1', 128),
  ('model_relu', 0),
  ('model_maxpool', 0),
  ('model_layer1_0_conv1', 36864),
  ('model_layer1_0_bn1', 128),
  ('model_layer1_0_relu', 0),
  ('model_layer1_0_conv2', 36864),
  ('model_layer1_0_bn2', 128)])

In [7]:
# getting subset to evaulate mean
edges, weights, a, b = module_to_graph(ResNet18())
# edges, weights
edges[:10], list(weights.items())[:10]



train_dl_limit, _ = get_dataloaders([0,1,2,3,4,5,6,7], train_limit=256)# 256
len(train_dl_limit)

Using cache found in /Users/konstantinakovlev/.cache/torch/hub/pytorch_vision_v0.10.0


Files already downloaded and verified
Files already downloaded and verified


16

In [8]:
inter = {}
model = ResNet18(8)
model.load_state_dict(torch.load('../data/model_last.ckpt', map_location='cpu'))
tr = torch.fx.symbolic_trace(model)
elem_count = 0
for x,_ in train_dl_limit:
    elem_count += x.shape[0]
    i_ = get_inter(tr, x)
    for k in i_:
        try:
            if k not in inter:
                inter[str(k)] = i_[k].sum(0).detach()
            else:
                inter[str(k)] += i_[k].sum(0).detach()
        except:
            print ('bad inter', k)
            #inter[str(k)] = 0.0
    for k in inter:
        inter[k] /= elem_count

Using cache found in /Users/konstantinakovlev/.cache/torch/hub/pytorch_vision_v0.10.0


bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1
bad inter 1


In [9]:
# likelihood of the model without pruning
model = ResNet18(8)
model.load_state_dict(torch.load('../data/model_last.ckpt', map_location='cpu'))

    
# full_ll = test_loop(model, train_dl_limit, nc=8, return_ll=True, device='cpu')
full_ll = test_loop(model, train_dl_limit, nc=8, return_ll=False, device='cpu')
full_ll

Using cache found in /Users/konstantinakovlev/.cache/torch/hub/pytorch_vision_v0.10.0


  0%|          | 0/16 [00:00<?, ?it/s]

0.79296875

In [10]:
# ll for pruned models
edge_ll = {}
model = ResNet18(8)
model.load_state_dict(torch.load('../data/model_last.ckpt', map_location='cpu'))

wrapped = torch.fx.symbolic_trace(model)
    
    
for e in tqdm(edges):
    pruned = PrunedModel(wrapped, {k:1.0 if k != e else 0.0 for k in edges }, inter)
    # edge_ll[e] = test_loop(pruned, train_dl_limit, nc=8, return_ll=True, device='cpu')
    edge_ll[e] = test_loop(pruned, train_dl_limit, nc=8, return_ll=False, device='cpu')
    

Using cache found in /Users/konstantinakovlev/.cache/torch/hub/pytorch_vision_v0.10.0


  0%|          | 0/78 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [20]:
# don't process logit edge, so setting its ll to min
edge_ll[('model_fc', 'output')] = max(edge_ll.values())
# edges_importance = [edge_ll[e]/max(edge_ll.values()) for e in edges]
edges_importance = [-edge_ll[e] / full_ll for e in edges]
edges_importance = [imp - min(edges_importance) for imp in edges_importance]

edges_importance[:10]
# {e : -edge_ll[e] / full_ll for e in edges}

[0.896551724137931,
 0.896551724137931,
 0.896551724137931,
 0.896551724137931,
 0.5862068965517241,
 0.5862068965517241,
 0.1280788177339901,
 0.1280788177339901,
 0.1280788177339901,
 0.1428571428571428]

## The second attempt + deleting of edges

1. Find all (sample) topological sorts

https://www.geeksforgeeks.org/all-topological-sorts-of-a-directed-acyclic-graph/

2. Apply greedy dynamic programming to find a monotonous solution

3. Postprocess a graph: remove all nodes that are unreacheble from "x".


In [12]:
DG = nx.DiGraph(edges)
all_sorts = list(nx.all_topological_sorts(DG))
len(all_sorts)

9261

In [13]:
len(edges), len(edges_importance)

(78, 78)

In [14]:
# @njit
def dp_for_top_sort(edges, weights, e_importance, top_sort_str, memory=1e10):
    node_ids = {k: i for i, k in enumerate(weights)}
    id_to_node = [node for _, node in enumerate(weights)]
    top_sort = np.array([node_ids[n] for n in top_sort_str])
    assert top_sort[-1] == node_ids['output']
    assert top_sort[0] == node_ids['x']
    assert top_sort.shape[0] == len(node_ids)
    m = np.zeros((len(node_ids), len(node_ids))).astype(np.int32)
    id_to_weight = np.array([weights[n] for n in id_to_node])
    assert len(edges) == len(e_importance)
    for (src, dst), w in zip(edges, e_importance):
        src_id, dst_id = node_ids[src], node_ids[dst]
        m[src_id, dst_id] = w
        
    node_to_layers = np.ones((len(node_ids), len(node_ids))).astype(np.int32) * (-100)  # ans for each v ->
    node_to_layers[top_sort[-1], top_sort[-1]] = 0
    dp = [1e9] * len(node_ids)
    dp[top_sort[-1]] = 0
    for i in range(len(node_ids) - 2, -1, -1):
        v = top_sort[i]
        for j in range(i, len(node_ids)):  # the last node of the first layer (starting from v)
            if id_to_weight[top_sort[i: j + 1]].sum() > memory:
                continue
            if j == len(node_ids) - 1:
                dp[v] = 0
                node_to_layers[v, top_sort[i:]] = 0
                continue
            v_j = top_sort[j + 1]
            next_layer_ids = [] if j + 1 >= len(node_ids) else \
            [k for k in range(m.shape[0]) if node_to_layers[v_j, k] == node_to_layers[v_j, v_j]]
            pruned_value = sum([m[top_sort[k], l] for k in range(i, j + 1) for l in next_layer_ids if m[top_sort[k], l] != 0])
            if dp[v_j] + pruned_value <= dp[v]:
                dp[v] = dp[v_j] + pruned_value
                node_to_layers[v] = node_to_layers[v_j]
                node_to_layers[v, top_sort[i:j + 1]] = node_to_layers[v].max() + 1
                
    # prune restricted edges (TODO: also prune unreacheble nodes)
    ans = node_to_layers[node_ids['x']]
    pruned_edges_ids = [(i, j) for i in range(m.shape[0]) for j in range(m.shape[0]) \
                    if m[i, j] != 0 and abs(ans[i] - ans[j]) > 1]
    pruned_edges = [(id_to_node[i], id_to_node[j]) for i, j in pruned_edges_ids]
    pruned_value = sum([m[i, j] for i, j in pruned_edges_ids])
    
    reach_ids = [set() for _ in range(len(node_ids))]
    for i in range(len(node_ids) - 1, -1, -1):
        v = top_sort[i]
        reach_ids[v].add(v)
        for k in range(i + 1, m.shape[0]):
            v_c = top_sort[k]
            if m[v, v_c] != 0 and (v, v_c) not in pruned_edges_ids:
                reach_ids[v] |= reach_ids[v_c]
    conn_g = top_sort[-1] in reach_ids[top_sort[0]]
                
    return {'node_to_layer': {id_to_node[i]: ans.max() - l for i, l in enumerate(ans)},
            'pruned_value': pruned_value, 'pruned_edges': pruned_edges,
            'connected_graph': conn_g}



In [18]:
### find the best solution
# import pickle
# with open('./naive_mean.pckl', 'rb') as inp:
#     eval_dict, fine_dict = pickle.loads(inp.read())
eval_dict = {}
fine_dict = {}

for edge_mem in range(2,6):
    if edge_mem in eval_dict and edge_mem in fine_dict:
        print ('skip', edge_mem)
        continue
    #if edge_mem == 3 and attemp > 0:
    #    continue
    #if edge_mem == 4 and attemp > 0:
    #    continue

    model = ResNet18(8)
    model.load_state_dict(torch.load('../data/model_last.ckpt', map_location='cpu'))
    warp = GraphInterperterWithGamma(model)

    named_dict = dict(model.named_modules())

    for node in warp.graph.nodes:
        if node.op == 'call_module':
            pass
            # print('Norm', np.sqrt(sum([(p ** 2).sum().item() \
            #                            for p in named_dict[node.target].parameters()])))
        # print(node.op, node.name, node.args)
        # break


    edges, weights, a, b = module_to_graph(ResNet18())
    # edges, weights
    edges[:10], list(weights.items())[:10]
    # edges_importance = [1]*len(edges)  ### WHY???
    print(edges_importance)

    best_pruned = 1e10
    best_val = None
    for s in tqdm(all_sorts):
        # for mem=8 the computation takes time
        res = dp_for_top_sort(edges, {k: 1 for k in weights}, edges_importance, s, edge_mem)  
        if res['connected_graph'] == True and best_pruned > res['pruned_value'] or best_val is None:
            best_pruned = res['pruned_value']
            best_val = res

        if best_pruned == 0:
            print(res)
            break


    model = ResNet18(8)
    model.load_state_dict(torch.load('../data/model_last.ckpt', map_location='cpu'))

    wrapped = torch.fx.symbolic_trace(model)
    pruned = PrunedModel(wrapped, {k:1.0 if k not in best_val['pruned_edges'] else 0.0 for k in edges }, inter)

    res = test_loop(pruned, test_dl,  "cpu", nc=8)
    if edge_mem not in eval_dict:
        eval_dict[edge_mem] = []
    eval_dict[edge_mem].append(res)

    train_loop(pruned, train_dl, test_dl, 9999999999, 1, 1e-3,  "cpu")
    res = test_loop(pruned, test_dl,  "cpu", nc=8)
    if edge_mem not in fine_dict:
        fine_dict[edge_mem] = []
    fine_dict[edge_mem].append(res)

    import pickle
    with open('naive_mean.pckl', 'wb') as out:
        out.write(pickle.dumps([eval_dict, fine_dict]))


Using cache found in /Users/konstantinakovlev/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/konstantinakovlev/.cache/torch/hub/pytorch_vision_v0.10.0


[0.896551724137931, 0.896551724137931, 0.896551724137931, 0.896551724137931, 0.5862068965517241, 0.5862068965517241, 0.1280788177339901, 0.1280788177339901, 0.1280788177339901, 0.1428571428571428, 0.6896551724137931, 0.8866995073891626, 0.2068965517241379, 0.2068965517241379, 0.029556650246305383, 0.029556650246305383, 0.029556650246305383, 0.024630541871921152, 0.7487684729064039, 0.8374384236453202, 0.5073891625615763, 0.5073891625615763, 0.3300492610837438, 0.3300492610837438, 0.3300492610837438, 0.3497536945812808, 0.3497536945812808, 0.3497536945812808, 0.18719211822660098, 0.8866995073891626, 0.044334975369458074, 0.044334975369458074, 0.029556650246305383, 0.029556650246305383, 0.029556650246305383, 0.024630541871921152, 0.8620689655172413, 0.8866995073891626, 0.3694581280788177, 0.3694581280788177, 0.3448275862068966, 0.3448275862068966, 0.3448275862068966, 0.15270935960591125, 0.15270935960591125, 0.3793103448275862, 0.054187192118226535, 0.8620689655172413, 0.1970443349753694

  0%|          | 0/9261 [00:00<?, ?it/s]

{'node_to_layer': {'x': 0, 'model_conv1': 0, 'model_bn1': 1, 'model_relu': 1, 'model_maxpool': 2, 'model_layer1_0_conv1': 2, 'model_layer1_0_bn1': 3, 'model_layer1_0_relu': 3, 'model_layer1_0_conv2': 4, 'model_layer1_0_bn2': 4, 'add': 5, 'model_layer1_0_relu_1': 5, 'model_layer1_1_conv1': 6, 'model_layer1_1_bn1': 6, 'model_layer1_1_relu': 7, 'model_layer1_1_conv2': 7, 'model_layer1_1_bn2': 8, 'add_1': 8, 'model_layer1_1_relu_1': 9, 'model_layer2_0_conv1': 10, 'model_layer2_0_bn1': 11, 'model_layer2_0_relu': 11, 'model_layer2_0_conv2': 12, 'model_layer2_0_bn2': 12, 'model_layer2_0_downsample_0': 9, 'model_layer2_0_downsample_1': 10, 'add_2': 13, 'model_layer2_0_relu_1': 13, 'model_layer2_1_conv1': 14, 'model_layer2_1_bn1': 14, 'model_layer2_1_relu': 15, 'model_layer2_1_conv2': 15, 'model_layer2_1_bn2': 16, 'add_3': 16, 'model_layer2_1_relu_1': 17, 'model_layer3_0_conv1': 18, 'model_layer3_0_bn1': 19, 'model_layer3_0_relu': 19, 'model_layer3_0_conv2': 20, 'model_layer3_0_bn2': 20, 'model

Using cache found in /Users/konstantinakovlev/.cache/torch/hub/pytorch_vision_v0.10.0


...

{'node_to_layer': {'x': 0, 'model_conv1': 0, 'model_bn1': 0, 'model_relu': 0, 'model_maxpool': 0, 'model_layer1_0_conv1': 1, 'model_layer1_0_bn1': 1, 'model_layer1_0_relu': 1, 'model_layer1_0_conv2': 1, 'model_layer1_0_bn2': 1, 'add': 2, 'model_layer1_0_relu_1': 2, 'model_layer1_1_conv1': 2, 'model_layer1_1_bn1': 2, 'model_layer1_1_relu': 2, 'model_layer1_1_conv2': 3, 'model_layer1_1_bn2': 3, 'add_1': 3, 'model_layer1_1_relu_1': 3, 'model_layer2_0_conv1': 4, 'model_layer2_0_bn1': 4, 'model_layer2_0_relu': 4, 'model_layer2_0_conv2': 4, 'model_layer2_0_bn2': 5, 'model_layer2_0_downsample_0': 3, 'model_layer2_0_downsample_1': 4, 'add_2': 5, 'model_layer2_0_relu_1': 5, 'model_layer2_1_conv1': 5, 'model_layer2_1_bn1': 5, 'model_layer2_1_relu': 6, 'model_layer2_1_conv2': 6, 'model_layer2_1_bn2': 6, 'add_3': 6, 'model_layer2_1_relu_1': 6, 'model_layer3_0_conv1': 7, 'model_layer3_0_bn1': 7, 'model_layer3_0_relu': 7, 'model_layer3_0_conv2': 8, 'model_layer3_0_bn2': 8, 'model_layer3_0_downsample

Using cache found in /Users/konstantinakovlev/.cache/torch/hub/pytorch_vision_v0.10.0


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [21]:
fine_dict

{2: [0.7201250195503235],
 3: [0.7294999957084656],
 4: [0.7146250009536743],
 5: [0.7152500152587891]}

In [24]:
eval_dict

{2: [0.7331249713897705],
 3: [0.7331249713897705],
 4: [0.7331249713897705],
 5: [0.7331249713897705]}

In [23]:
test_loop(model, test_dl, nc=8, return_ll=False, device='cpu')


  0%|          | 0/500 [00:00<?, ?it/s]

0.7152500152587891

In [67]:
if edge_mem not in eval_dict:
    eval_dict[edge_mem] = []
eval_dict[edge_mem].append(res)

train_loop(pruned, train_dl, test_dl, 9999999999, 1, 1e-3,  "cpu")
res = test_loop(pruned, test_dl,  "cpu", nc=8)
if edge_mem not in fine_dict:
    fine_dict[edge_mem] = []
fine_dict[edge_mem].append(res)

import pickle
with open('naive.pckl', 'wb') as out:
    out.write(pickle.dumps([eval_dict, fine_dict]))


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [2]:
import pickle
with open('naive.pckl', 'rb') as inp:
    data = pickle.loads(inp.read())
data

[{2: [0.125], 3: [0.125], 4: [0.12524999678134918], 5: [0.23675000667572021]},
 {2: [0.5216249823570251],
  3: [0.5532500147819519],
  4: [0.656624972820282],
  5: [0.6958749890327454]}]

In [2]:
import pickle
with open('randn.pckl', 'rb') as inp:
    data = pickle.loads(inp.read())
data

[{2: [0.125, 0.12600000202655792, 0.125],
  3: [0.125, 0.125, 0.125],
  4: [0.12524999678134918, 0.12524999678134918, 0.125],
  5: [0.1368750035762787, 0.125, 0.21812500059604645]},
 {2: [0.5640000104904175, 0.5180000066757202, 0.5730000138282776],
  3: [0.5625, 0.5435000061988831, 0.5742499828338623],
  4: [0.6353750228881836, 0.6573749780654907, 0.6784999966621399],
  5: [0.6420000195503235, 0.6677500009536743, 0.6775000095367432]}]

In [16]:
import pickle
with open('naive_mean.pckl', 'rb') as inp:
    data = pickle.loads(inp.read())
data

[{2: [0.125], 3: [0.125], 4: [0.1264999955892563], 5: [0.2516250014305115]},
 {2: [0.5951250195503235],
  3: [0.6156250238418579],
  4: [0.6588749885559082],
  5: [0.6837499737739563]}]

In [5]:
import pickle
with open('random_mean.pckl', 'rb') as inp:
    data = pickle.loads(inp.read())
data

[{2: [0.12962499260902405, 0.125, 0.125],
  3: [0.125, 0.125, 0.125],
  4: [0.12612499296665192, 0.12612499296665192, 0.125],
  5: [0.12587499618530273, 0.12587499618530273]},
 {2: [0.5892500281333923, 0.5241249799728394, 0.6132500171661377],
  3: [0.5706250071525574, 0.6041250228881836, 0.5644999742507935],
  4: [0.6047499775886536, 0.6507499814033508, 0.6359999775886536],
  5: [0.6508749723434448, 0.6942499876022339]}]